In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
sns.set_style('whitegrid')

In [ ]:
np.random.seed(1767)

Your task is to predict the Site EUI for each row, given the characteristics of the building and the weather data for the location of the building. 

In [ ]:
df = pd.read_csv('./data/train.csv.zip').drop(columns=['id'])

In [ ]:
test_df = pd.read_csv('C:/Users/k202141/Downloads/test.csv.zip').drop(columns=['id'])

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
.describe()

In [ ]:
test_df.clip().describe()

In [ ]:
cat_columns = list(df.select_dtypes(['object']).columns)
cat_columns

In [ ]:
for cc in cat_columns:
    df[f'{cc}_'] = df[cc].astype('category').cat.codes
    test_df[f'{cc}_'] = test_df[cc].astype('category').cat.codes    
df = df.drop(columns=cat_columns)

In [ ]:
df.info()

In [ ]:
df.corr()['site_eui'].sort_values().head()
df.corr()['site_eui'].sort_values().tail()

In [ ]:
df.corr()['energy_star_rating'].sort_values().tail()

## Data exploration

In [ ]:
sns.scatterplot(data=df, x='floor_area', y='site_eui')

In [ ]:
C = df.corr()

In [ ]:
sns.heatmap(np.abs(C))
ax=plt.gca()
ax.set_aspect('equal')

In [ ]:
mean_train = df.drop(columns=['site_eui']).describe().loc['mean']
mean_test  = test_df.describe().loc['mean']

In [ ]:
plt.plot(mean_test / mean_train)

In [ ]:
tmp = ((mean_test / mean_train - 1).abs() < 0.1)
tmp[tmp==1].index

In [ ]:
len(test_df.columns), len(df.columns)

In [ ]:
df.isna().sum().sort_values(ascending=False).head(30)

In [ ]:
df['Year_Factor'].unique()

In [ ]:
test_df['Year_Factor'].describe()